# Run the other two notebooks first before running this!

This notebook assumes that you've 1) downloaded the initial dataset, and 2) ran the Branded Food Initial Data Cleaning notebook. Running this with pgadmin will populate the dataset with the cleaned datafiles.

### Tables to create:
- branded_food_reduced -> branded_foods table (main food data file) (done)
- brand_names -> brand_names table (normalization for brand names)
- brand_owners -> brand_owners table
- food_categories -> food_categories table
- ingredients -> ingredients table
- food_to_id -> f_to_ing table (food to ingredient for ingredients, should be made after ingredients)

In [48]:
import os
import psycopg2
import pandas as pd
import psycopg2.extras as extras 
from sql_creation import run_command, connect, create_insert_branded_foods
import numpy as np

In [38]:
# Create initial connection to the database
conn, cursor = connect()
conn, cursor

Attempting to connect to postgreSQL database...
Connected
postgreSQL db version: ('PostgreSQL 16.2, compiled by Visual C++ build 1937, 64-bit',)


(<connection object at 0x0000025983352E10; dsn: 'user=postgres password=xxx dbname=food_db host=localhost', closed: 0>,
 <cursor object at 0x000002599B0A2F20; closed: 0>)

In [39]:
create_insert_branded_foods(conn, clear_table=True)

C:\Users\Alexander\Desktop\FDA-Data-Analysis\sql_creation.py:57: DtypeWarning: Columns (2,5,9,10,31) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_food = pd.read_csv(os.path.join('cleaned', 'branded_food_reduced.csv'))#.drop(columns='Unnamed: 0')


<class 'psycopg2.errors.DuplicateTable'> relation "branded_foods" already exists



In [58]:
def create_brand_names(clear_table=True):
    brand_names = pd.read_csv(os.path.join('cleaned', 'brand_names.csv'))[['brand_name_id', 'brand_name']]
    brand_names.head()
    
    # Create table
    brand_names_schema = """
        CREATE TABLE brand_names (
            brand_id INTEGER PRIMARY KEY NOT NULL,
            brand_name VARCHAR(64) NOT NULL
        )
    """

    run_command(conn, brand_names_schema)
    
    # Clear table if needed
    if clear_table:
        run_command(conn, 'DELETE FROM brand_names')
    
    # Insert values into table
    insert_command = "INSERT INTO brand_names VALUES (%s, %s)"
    
    for row in brand_names.to_numpy():
        run_command(conn, insert_command, row)

In [67]:
clear_table = True
brand_owners = pd.read_csv(os.path.join('cleaned', 'brand_owners.csv'))[['brand_owner_id', 'brand_owner']]
brand_owners.head()

# Create table
brand_owners_schema = """
    CREATE TABLE brand_owners (
        brand_id INTEGER PRIMARY KEY NOT NULL,
        brand_name VARCHAR(128) NOT NULL
    )
"""

run_command(conn, brand_owners_schema)

# Clear table if needed
if clear_table:
    run_command(conn, 'DELETE FROM brand_owners')

# Insert values into table
insert_command = "INSERT INTO brand_owners VALUES (%s, %s)"

for row in brand_names.to_numpy():
    run_command(conn, insert_command, row)